In [41]:
import pylab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
flow = pd.read_csv("flow.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
occupancy = pd.read_csv("occupancy.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
speed = pd.read_csv("speed.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
timestamp = pd.read_csv("timestamp.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)

KeyboardInterrupt: 

In [ ]:
flow.head()

In [ ]:
df = pd.concat([flow,occupancy,speed,timestamp], axis=1)

In [ ]:
df.columns = ['flow1','flow2','flow3','flow4','occupancy1','occupancy2', 'occupancy3','occupancy4','speed1','speed2', 'speed3','speed4','timestamp']
df.head()

In [ ]:
df1 =  pd.DataFrame({"flow" : df.flow1, "occupancy" : df.occupancy1,"speed" : df.speed1,"timestamp" : df.timestamp,"detector" : "x"})
df2 = pd.DataFrame({"flow" : df.flow2, "occupancy" : df.occupancy2,"speed" : df.speed2,"timestamp" : df.timestamp,"detector" : "y"})
df3 = pd.DataFrame({"flow" : df.flow3, "occupancy" : df.occupancy3,"speed" : df.speed3,"timestamp" : df.timestamp,"detector" : "z"})
df4 = pd.DataFrame({"flow" : df.flow4, "occupancy" : df.occupancy4,"speed" : df.speed4,"timestamp" : df.timestamp,"detector" : "x1"})

In [ ]:
frames = [df1, df2, df3, df4]
result = pd.concat(frames)

In [ ]:
result = result.sort_values("timestamp")

In [ ]:
len(result)

In [ ]:
# Finding outlier values for Occoupancy
val = np.sort(result["occupancy"].unique())
plt.plot(val)

In [ ]:
print (np.sort(result["occupancy"].unique()))

In [ ]:
# Outlier speed visualization
plt.figure(figsize=(10,10))
plt.plot(np.sort(result["occupancy"].unique()))

In [ ]:
removeResult1 = result[(result['speed'] < 0) | (result['flow'] < 0)]
removeResult2 = result[(result['flow'] == 0) & (result['speed'] > 0)]
jam_occupancy_threshold = 100
removeResult3 = result[((result['speed'] == 0) & 
                            (result['flow'] == 0) & 
                            (result['occupancy'] != 0) & 
                            (result['occupancy'] < jam_occupancy_threshold))]
# skewed data
#removeResult4 = result[(result['speed'] == 0) & (result['flow'] == 0) & (result["occupancy"] == 0)]
removeResult5 = result[(result['speed'] == 0) & (result['flow'] != 0) & (result["occupancy"] == 0)]
removeResult6 = result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]

In [ ]:
len(result)

In [ ]:
result = pd.concat([result,removeResult1,removeResult1]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult2,removeResult2]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult3,removeResult3]).drop_duplicates(keep=False)
#result = pd.concat([result,removeResult4,removeResult4]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult5,removeResult5]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult6,removeResult6]).drop_duplicates(keep=False)

In [ ]:
result = result.sort_values(["flow"])

In [ ]:
def sampleData(data, threshold, columnName):
    flowBins = np.zeros(shape=(20,), dtype = object)
    data = data.sort_values(columnName)
    difference = threshold
    high  = difference
    low = 0;
    for i in range(0, 20, 1):
        b = data[data[columnName] < high]
        b = b[b[columnName] >= low]
        flowBins[i] = b
        low = high
        high = high + difference
    return flowBins

In [ ]:
print (len(result))

In [ ]:
flowBins = np.zeros(shape=(20,), dtype = object)
flowBins = sampleData(result, 13 ,"flow")    

In [ ]:

flow_occupancy_bin = np.ndarray(shape=(20,20), dtype = object)
for i in range(0,20,1):
    data = pd.DataFrame(flowBins[i])
    flow_occupancy_bin[i] = sampleData(data, 6 ,"occupancy") 

In [ ]:
flow_occupancy_speed_bin = np.ndarray(shape=(20,20,20), dtype = object)
for i in range(0,20,1):
    for j in range(0,20,1):
        data = flow_occupancy_bin[i][j];
        flow_occupancy_speed_bin[i][j] = sampleData(data, 11 ,"speed") 

In [ ]:
!mkdir output

In [ ]:
final_result = pd.DataFrame(columns = ["detector","flow","occupancy","speed","timestamp", "probability"])
total_points = len(result)
for i in range(0,20,1):
    for j in range(0,20,1):
        for k in range(0,20,1):
            prob_data = flow_occupancy_speed_bin[i][j][k]
            points_inside_box = prob_data.shape[0]
            f = prob_data.flow.max() - prob_data.flow.min()
            o = prob_data.occupancy.max() - prob_data.occupancy.min()
            s = prob_data.speed.max() - prob_data.speed.min()
            volume = 858
            if(volume == 0): volume =1
            probability = points_inside_box/float(total_points * volume)
            prob_data["probability"] = probability
            prob_data.to_csv("output/out" + str(i) + '-' + str(j) + '-' + str(k) + ".csv",sep="\t")

In [ ]:
! cat output/*.csv > merged.csv

In [ ]:
!rm -rf output/

In [ ]:
merged = pd.read_csv("merged.csv", na_values=['-'], delimiter="\t", error_bad_lines=False)

In [ ]:
removeHeader = merged[merged["probability"] == "probability"]

In [ ]:
merged = pd.concat([merged,removeHeader,removeHeader]).drop_duplicates(keep=False)

In [ ]:
removeResult1["probability"] = 0
removeResult2["probability"] = 0
removeResult3["probability"] = 0
#removeResult4["probability"] = 0
removeResult5["probability"] = 0
removeResult6["probability"] = 0

In [ ]:
merged = merged.append(removeResult1);
merged = merged.append(removeResult2);
merged = merged.append(removeResult3);
#merged = merged.append(removeResult4);
merged = merged.append(removeResult5);
merged = merged.append(removeResult6);

In [ ]:
# merged = merged.reset_index(drop=True)
merged["probability"] = merged["probability"].apply(pd.to_numeric)
merged = merged.sort_values(["probability"])
final_columns = ['flow', 'speed', 'occupancy', 'probability']
merged = merged[final_columns]

In [ ]:
# merged
final_result = pd.DataFrame(columns = ["flow","speed","occupancy", "probability"])
for i in range(0,len(merged),100):
    final_result = final_result.append(merged.iloc[[i]])

In [ ]:
final_result.to_csv("3445.txt",sep="\t", index=False)

In [44]:
low = 0;
high1 = int(len(result)/20)
BinNum = 20;
length = high1 - low
flowBins = np.zeros(shape=(20,high1), dtype = object)
for i in range(0, 20, 1):
    high = low + high1
    flowBins[i] = [result[low : high]]
    low = high

In [45]:
import numpy as np
flow_occupancy_bin = np.ndarray(shape=(20,20), dtype = object)
y = 1;
low = 0;
for i in range(0,20,1):
    data = pd.DataFrame(flowBins[i].reshape((-1,length))[0][0]);
    data = data.sort_values(["occupancy"])
    high1 = int(len(flowBins[1])/20)
    low = 0
    for j in range(0, 20, 1):
        high = low + high1
        flow_occupancy_bin[i][j] = [data[low : high]]
        low = high

In [46]:
import numpy as np
flow_occupancy_speed_bin = np.ndarray(shape=(20,20,20), dtype = object)
for i in range(0,20,1):
    for k in range(0,20,1):
        data = flow_occupancy_bin[i][k][0];
        data = data.sort_values(["speed"])
        low = 0;
        high1 = int(len(flow_occupancy_bin[0][0][0])/20)
        for j in range(0, 20, 1):
            high = low + high1
            flow_occupancy_speed_bin[i][k][j] = [data[low : high]]
            low = high

In [47]:
!mkdir output

In [48]:
vectors = len(result)
for i in range(1,20,1):
    for j in range(1,20,1):
        for k in range(1,20,1):
            prob_data = flow_occupancy_speed_bin[i][j][k][0]
            currVectors = flow_occupancy_speed_bin[i][j][k][0].shape[0]
            flow_difference = prob_data.flow.max() - prob_data.flow.min()
            occupancy_difference = prob_data.occupancy.max() - prob_data.occupancy.min()
            speed_difference = prob_data.speed.max() - prob_data.speed.min()
            volume = flow_difference * occupancy_difference * speed_difference
            if(volume == 0): volume =1
            probability = currVectors/float(vectors * volume)
            prob_data["probability"] = probability
            prob_data.to_csv("output/out" + str(i) + '-' + str(j) + '-' + str(k) + ".csv",sep="\t")  

/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [49]:
! cat output/*.csv > merged-1.csv

In [50]:
!rm -rf output/

In [51]:
merged = pd.read_csv("merged-1.csv", na_values=['-'], delimiter="\t", error_bad_lines=False)

In [52]:
removeHeader = merged[merged["probability"] == "probability"]
merged = pd.concat([merged,removeHeader,removeHeader]).drop_duplicates(keep=False)

In [53]:
removeResult1["probability"] = 0
removeResult2["probability"] = 0
removeResult3["probability"] = 0
#removeResult4["probability"] = 0
removeResult5["probability"] = 0
removeResult6["probability"] = 0

In [54]:
merged = merged.append(removeResult1);
merged = merged.append(removeResult2);
merged = merged.append(removeResult3);
#merged = merged.append(removeResult4);
merged = merged.append(removeResult5);
merged = merged.append(removeResult6);

In [ ]:
merged["probability"] = merged["probability"].apply(pd.to_numeric)
merged = merged.sort_values(["probability"])
final_columns = ['flow', 'speed', 'occupancy', 'probability']
merged = merged[final_columns]

In [ ]:
final_result = pd.DataFrame(columns = ["flow","speed","occupancy", "probability"])
for i in range(0,len(merged),100):
    final_result = final_result.append(merged.iloc[[i]])

In [ ]:
final_result.to_csv("3445_2.txt",sep="\t", index=False)